<h3>Setup</h3>

In [1]:
# Imports
import pandas as pd
import numpy as np

import kagglehub
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier

In [12]:
# Load Kaggle dataset
def load_kaggle_csv(dataset, filename):
    path = Path(kagglehub.dataset_download(dataset))
    return pd.read_csv(path / filename)

df = load_kaggle_csv(
    "chitwanmanchanda/fraudulent-transactions-data",
    "Fraud.csv"
)

# Sort by time
df = df.sort_values('step').reset_index(drop=True)

<h3>Feature Engineering</h3>

In [13]:
# Determine if merchant or customer
df["orig_is_customer"] = df["nameOrig"].str.startswith("C").astype("int8")
df["dest_is_customer"] = df["nameDest"].str.startswith("C").astype("int8")

In [14]:
# Calculate change in account value (customers only)
df["orig_delta"] = 0.0
df["dest_delta"] = 0.0

df.loc[df["orig_is_customer"] == 1, "orig_delta"] = (
    df.loc[df["orig_is_customer"] == 1, "newbalanceOrig"] -
    df.loc[df["orig_is_customer"] == 1, "oldbalanceOrg"]
)

df.loc[df["dest_is_customer"] == 1, "dest_delta"] = (
    df.loc[df["dest_is_customer"] == 1, "newbalanceDest"] -
    df.loc[df["dest_is_customer"] == 1, "oldbalanceDest"]
)

In [15]:
# Calculate outflow
df["orig_outflow"] = (-df["orig_delta"]).clip(lower=0)

In [19]:
# Calculate ratio
df["orig_amount_ratio"] = df["amount"] / (df["oldbalanceOrg"])
df["dest_amount_ratio"] = df["amount"] / (df["oldbalanceDest"])

In [20]:
df

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,...,dest_is_customer,orig_delta,dest_delta,orig_outflow,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,orig_amount_ratio,dest_amount_ratio
0,1,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0,...,0,-9839.64,0.00,9839.64,0,0,1,0,0.057834,inf
1,1,5157.05,C1562950869,1667.92,0.00,M2021835850,0.00,0.00,0,0,...,0,-1667.92,0.00,1667.92,0,0,1,0,3.091905,inf
2,1,5746.44,C845388562,0.00,0.00,M550572371,0.00,0.00,0,0,...,0,0.00,0.00,-0.00,0,0,1,0,inf,inf
3,1,5607.36,C948424584,5202.00,0.00,M1447685190,0.00,0.00,0,0,...,0,-5202.00,0.00,5202.00,0,0,1,0,1.077924,inf
4,1,6360.79,C2027701910,3731.00,0.00,M1345293143,0.00,0.00,0,0,...,0,-3731.00,0.00,3731.00,0,0,1,0,1.704849,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,C2013999242,339682.13,0.00,C1850423904,0.00,0.00,1,0,...,1,-339682.13,0.00,339682.13,0,0,0,1,1.000000,inf
6362616,743,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0,...,1,-339682.13,339682.13,339682.13,1,0,0,0,1.000000,inf
6362617,743,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0,...,1,-6311409.28,0.00,6311409.28,0,0,0,1,1.000000,inf
6362618,743,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0,...,1,-6311409.28,6311409.27,6311409.28,1,0,0,0,1.000000,92.152375


In [16]:
# Apply one hot encording for transaction type
df = pd.get_dummies(df, columns=['type'], drop_first=True)

In [9]:
# Drop name and flag columns
df = df.drop(columns=[
    'nameOrig', 'nameDest',
    'isFlaggedFraud'
])

<h3>Machine Learning</h3>

In [ ]:
X = df.drop('isFraud', axis=1)
y = df['isFraud']

split_index = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

print(f"X_train shape {X_train.shape}")
print(f"X_test shape {X_test.shape}")

In [ ]:
X_train

In [ ]:
model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
    eval_metric='aucpr',
    use_label_encoder=False,
    n_jobs=-1
)

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))